In [1]:
import os
import torch
from torch.utils.data import DataLoader, Dataset, random_split
from torchvision import transforms
from PIL import Image
import pandas as pd
from sklearn.model_selection import train_test_split
import torch.nn as nn
import torchvision.models as models
import numpy as np
import torch.optim as optim
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
from torchinfo import summary
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score

# **data loading & processing**

In [2]:
images_dir = "/kaggle/input/ham1000-segmentation-and-classification/images" 
masks_dir = "/kaggle/input/ham1000-segmentation-and-classification/masks"    
csv_file = "/kaggle/input/ham1000-segmentation-and-classification/GroundTruth.csv"

In [ ]:
data = pd.read_csv(csv_file)

# Ensure labels are numeric
disease_names = ['MEL', 'NV', 'BCC', 'AKIEC', 'BKL', 'DF', 'VASC']
data[disease_names] = data[disease_names].apply(pd.to_numeric, errors='coerce')

# Data Visualization
plt.figure(figsize=(10, 6))
data[disease_names].sum().plot(kind='bar', color='skyblue', edgecolor='black')
plt.title('Count of Each Disease')
plt.ylabel('Count')
plt.xlabel('Diseases')
plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

In [4]:
dataset = []
for idx, row in data.iterrows():
    image_id = row['image']
    label = torch.tensor(row[disease_names].values.astype(float), dtype=torch.float32)
    img_path = os.path.join(images_dir, f"{image_id}.jpg")
    mask_path = os.path.join(masks_dir, f"{image_id}_segmentation.png")
    dataset.append((img_path, mask_path, label))

# Split Data
train_data, test_data = train_test_split(dataset, test_size=0.2, random_state=42)
train_data, val_data = train_test_split(train_data, test_size=0.25, random_state=42)

In [5]:
transform = {
    "image": transforms.Compose([
        transforms.Resize((128, 128)),
        transforms.ToTensor()
    ]),
    "mask": transforms.Compose([
        transforms.Resize((128, 128)),
        transforms.ToTensor()
    ])
}

In [6]:
# Ensure labels are binary
def process_sample(data_entry, transform):
    img_path, mask_path, label = data_entry
    image = transform["image"](Image.open(img_path).convert("RGB"))
    mask = transform["mask"](Image.open(mask_path).convert("L"))
    
    label = (label > 0).float()  # Ensure binary labels (0 or 1)
    
    return image, mask, label

# Create Dataloader function
def create_dataloader(data, batch_size, shuffle):
    processed_data = [process_sample(entry, transform) for entry in data]
    images, masks, labels = zip(*processed_data)
    return DataLoader(torch.utils.data.TensorDataset(torch.stack(images), torch.stack(masks), torch.stack(labels)), batch_size=batch_size, shuffle=shuffle)

In [7]:
batch_size = 16
train_loader = create_dataloader(train_data, batch_size, True)
val_loader = create_dataloader(val_data, batch_size, False)
test_loader = create_dataloader(test_data, batch_size, False)


In [ ]:
def show_samples(dataloader, title):
    batch = next(iter(dataloader))
    images, masks, labels = batch
    fig, axes = plt.subplots(1, 5, figsize=(15, 5))
    fig.suptitle(title)
    for i in range(5):
        img = images[i].permute(1, 2, 0).numpy()
        lbl = labels[i].numpy()
        axes[i].imshow(img)
        axes[i].set_title(f"Label: {np.argmax(lbl)}")
        axes[i].axis("off")
    plt.show()

# Show samples
show_samples(train_loader, "Train Samples")
show_samples(val_loader, "Validation Samples")
show_samples(test_loader, "Test Samples")

# model loading and training

In [9]:
model = models.resnet50(pretrained=True)
num_features = model.fc.in_features  # Fix: Define num_features
num_classes = 7

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 209MB/s]


In [10]:
for param in model.parameters():
    param.requires_grad = False  # Freeze all layers

for param in model.fc.parameters():
    param.requires_grad = True  # Train only final layers


In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


calculating class weights to help with imbalance

focal loss instead with BCEwithLogits to help also with imbalance

In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.utils.class_weight import compute_class_weight

# Compute class weights
all_labels = [torch.argmax(label).item() for _, _, label in dataset]  # Extract class index
class_weights = compute_class_weight(class_weight="balanced", classes=np.unique(all_labels), y=all_labels)
class_weights = torch.tensor(class_weights, dtype=torch.float32).to(device)  # Convert to tensor
print("Class Weights:", class_weights)

class FocalLoss(nn.Module):
    def __init__(self, alpha=None, gamma=2.0, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, inputs, targets):
        BCE_loss = nn.functional.binary_cross_entropy_with_logits(inputs, targets, reduction='none')
        pt = torch.exp(-BCE_loss)  # Compute probability of correct classification
        focal_loss = (1 - pt) ** self.gamma * BCE_loss  # Apply focal term

        if self.alpha is not None:
            focal_loss *= self.alpha

        if self.reduction == 'mean':
            return focal_loss.mean()
        elif self.reduction == 'sum':
            return focal_loss.sum()
        else:
            return focal_loss

Class Weights: tensor([ 1.2855,  0.2134,  2.7835,  4.3753,  1.3018, 12.4410, 10.0755],
       device='cuda:0')


In [13]:
model.fc = nn.Sequential(
    nn.Linear(num_features, 512),
    nn.ReLU(),
    nn.Dropout(0.7),
    nn.Linear(512, num_classes) 
)

In [14]:
criterion = FocalLoss(alpha=class_weights, gamma=2, reduction='mean')

In [15]:
model = model.to(device)

In [16]:
optimizer = optim.Adam(model.parameters(), lr=0.0001, weight_decay=1e-3)

# Early stopping settings
early_stopping_patience = 5
epochs = 50
train_losses, val_losses = [], []

In [17]:
def train_and_validate(model, criterion, optimizer, train_loader, val_loader, epochs, patience, device):
    best_val_loss = float('inf')
    patience_counter = 0

    for epoch in range(epochs):
        model.train()
        train_loss = 0.0
        for inputs, _, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device).float()
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        # Validation
        val_loss = 0.0
        model.eval()
        with torch.no_grad():
            for inputs, _, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device).float()
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()

        train_losses.append(train_loss / len(train_loader))
        val_losses.append(val_loss / len(val_loader))
        print(f"Epoch {epoch+1}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")

        # Early Stopping
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), 'best_model.pth')
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print("Early stopping triggered!")
                break

In [18]:
train_and_validate(model, criterion, optimizer, train_loader, val_loader, epochs, early_stopping_patience, device)

Epoch 1, Train Loss: 65.1466, Val Loss: 16.7582
Epoch 2, Train Loss: 48.6328, Val Loss: 14.4355
Epoch 3, Train Loss: 43.7386, Val Loss: 14.1137
Epoch 4, Train Loss: 40.7794, Val Loss: 13.7917
Epoch 5, Train Loss: 40.2157, Val Loss: 12.5035
Epoch 6, Train Loss: 39.5193, Val Loss: 12.6365
Epoch 7, Train Loss: 37.1121, Val Loss: 12.5162
Epoch 8, Train Loss: 36.9226, Val Loss: 11.8736
Epoch 9, Train Loss: 36.0074, Val Loss: 13.0001
Epoch 10, Train Loss: 35.9032, Val Loss: 11.9224
Epoch 11, Train Loss: 34.3112, Val Loss: 11.6472
Epoch 12, Train Loss: 34.8901, Val Loss: 11.7152
Epoch 13, Train Loss: 34.9290, Val Loss: 11.7004
Epoch 14, Train Loss: 34.1246, Val Loss: 12.0438
Epoch 15, Train Loss: 34.2631, Val Loss: 12.3343
Epoch 16, Train Loss: 33.1542, Val Loss: 11.8796
Early stopping triggered!


In [ ]:

plt.figure(figsize=(6, 4))
plt.plot(train_losses, label="Train Loss", linestyle="-", color="blue")
plt.plot(val_losses, label="Validation Loss", linestyle="--", color="orange")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.title("Training and Validation Loss")
plt.show()


# Results

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score


def plot_confusion_matrix(y_true, y_pred, class_names):
    cm = confusion_matrix(np.argmax(y_true, axis=1), np.argmax(y_pred, axis=1))
    
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
    plt.xlabel("Predicted Labels")
    plt.ylabel("True Labels")
    plt.title("Confusion Matrix")
    plt.show()

def evaluate_model(model, test_loader, device):
    model.load_state_dict(torch.load("best_model.pth"))
    model.eval()
    
    y_true, y_pred = [], []
    
    with torch.no_grad():
        for inputs, _, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = torch.sigmoid(model(inputs))  # Convert logits to probabilities
            predicted = (outputs > 0.5).float()  # Convert to binary predictions
            
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(predicted.cpu().numpy())
    
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)

    accuracy = accuracy_score(np.argmax(y_true, axis=1), np.argmax(y_pred, axis=1))
    precision = precision_score(y_true, y_pred, average="weighted", zero_division=1)
    recall = recall_score(y_true, y_pred, average="weighted", zero_division=1)
    f1 = f1_score(y_true, y_pred, average="weighted", zero_division=1)

    print(f"Test Accuracy: {accuracy:.4f}")
    print(f"Test Precision: {precision:.4f}")
    print(f"Test Recall: {recall:.4f}")
    print(f"Test F1-score: {f1:.4f}")

    print("\nPer-Class Metrics:")
    print(classification_report(y_true, y_pred, target_names=disease_names, zero_division=1))

    plot_confusion_matrix(y_true, y_pred, disease_names)

evaluate_model(model, test_loader, device)
